# Run test application

## 1. Download sample data from Sentinel Data Hub

In [1]:
# Install python package sentinelsat
! pip install sentinelsat==0.13 --user

In [2]:
# connect to the API
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

### <span style="color:red">Create Account (https://scihub.copernicus.eu/dhus/#/self-registration) and change user and password below</span>.

In [ ]:
user = 'username'
password = 'password'

In [ ]:
# initialize settings
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/apihub/')

In [ ]:
# search by polygon (MNI test site coordinates), time, and SciHub query keywords
footprint = geojson_to_wkt(read_geojson('coordinates_mni.geojson'))
products = api.query(footprint,
                     date=('20200101', '20200120'),
                     platformname='Sentinel-1',
                     producttype='SLC')
print('Following products will be downloaded')
print(api.to_dataframe(products).title.values)

print('These {} product need {} Gb of disk space'.format(len(products), api.get_products_size(products)))

Start download process (attention: might take a while and will need free some disk space)

In [ ]:
import os

# download all results from the search
api.download_all(products,os.getcwd()+'/test_files')

## 2. Process SAR data

In [ ]:
input_folder = os.getcwd()+'/test_files'
output_folder = os.getcwd()+'/test_files'
gpt_location = '/home/tweiss/snap/bin/gpt'

In [ ]:
import yaml

with open('sample_config_file.yaml') as stream:
   data = yaml.safe_load(stream)

data['input_folder'] = input_folder
data['output_folder'] = output_folder
data['gpt'] = gpt_location

with open('test_config_file.yaml', 'wb') as stream:
   yaml.safe_dump(data, stream, default_flow_style=False, 
                  explicit_start=True, allow_unicode=True, encoding='utf-8')

In [ ]:
from sar_pre_processing.sar_pre_processor import *
import warnings
warnings.filterwarnings("ignore")

processing = SARPreProcessor(config='test_config_file.yaml')
processing.create_processing_file_list()
print('start step 1')
processing.pre_process_step1()
print('start step 2')
processing.pre_process_step2()
print('start step 3')
processing.pre_process_step3()
print('start solve projection problem')
processing.solve_projection_problem()
print('start add netcdf information')
processing.add_netcdf_information()
print('start create netcdf stack')
processing.create_netcdf_stack()